In [9]:
from transformer import Transformer
import torch
import numpy as np

ModuleNotFoundError: No module named 'transformer'

In [ ]:
fulfulde_file = '/Users/dmarshall/Burkina-Testing/corpus/en-fuh/parallel/fuh.txt'
english_file = '/Users/dmarshall/Burkina-Testing/corpus/en-fuh/parallel/en.txt'

START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

fulfulde_vocab = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', ',',
                'a', 'b', 'ɓ', 'mb', 'c', 'd', 'ɗ', 'nd', 'e', 'f', 'g', 'ng', 'h', 'i', 'j', 'nj', 'k', 'l',
                'm', 'n', 'ŋ', 'ñ', 'o', 'p', 'q', 'r', 's', 'sh', 't', 'u', 'w', 'x', 'y', 'ƴ', 'z',
                PADDING_TOKEN, END_TOKEN]

english_vocab = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', ',', '0', '1', '2',
                '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '@', '[', '\'', ']', '^', '_', '`',
                'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [ ]:
index_to_fulfulde = {k:v for k,v in enumerate(fulfulde_vocab)}
fulfulde_to_index = {v:k for k,v in enumerate(fulfulde_vocab)}
index_to_english = {k:v for k,v in enumerate(english_vocab)}
english_to_index = {v:k for k,v in enumerate(english_vocab)}

In [ ]:
with open(fulfulde_file, 'r') as file:
    fulfulde_sentences = file.readlines()
with open(english_file, 'r') as file:
    english_sentences = file.readlines()

TOTAL_SENTENCES = 100000
fulfulde_sentences = fulfulde_sentences[:TOTAL_SENTENCES]
english_sentences = english_sentences[:TOTAL_SENTENCES]

fulfulde_sentences = [sentence.rstrip('\n') for sentence in fulfulde_sentences]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]

In [ ]:
fulfulde_sentences[:10]

In [ ]:
english_sentences[:10]

In [ ]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Fulfulde: {np.percentile([len(x) for x in fulfulde_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


In [ ]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1)

valid_sentence_indices = []
for index in range(len(fulfulde_sentences)):
    fulfulde_sentence, english_sentence = fulfulde_sentences[index], english_sentences[index]
    if is_valid_length(fulfulde_sentence, max_sequence_length) \
        and is_valid_length(english_sentence, max_sequence_length) \
        and is_valid_tokens(fulfulde_sentence, fulfulde_vocab):
            valid_sentence_indices.append(index)

print(f"Number of sentences: {len(fulfulde_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indices)}")

In [ ]:
fulfulde_sentences = [fulfulde_sentences[i] for i in valid_sentence_indices]
english_sentences = [english_sentences[i] for i in valid_sentence_indices]

In [ ]:
fulfulde_sentences[:3]

In [ ]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
fuh_vocab_size = len(fulfulde_vocab)

transformer = Transformer(
    d_model,
    num_heads,
    ffn_hidden,
    drop_prob,
    num_layers,
    max_sequence_length,
    fuh_vocab_size,
    english_to_index,
    fulfulde_to_index,
    START_TOKEN,
    END_TOKEN,
    PADDING_TOKEN
)

In [10]:
transformer

NameError: name 'transformer' is not defined

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, english_sentences, fulfulde_sentences):
        self.fulfulde_sentences = fulfulde_sentences
        self.english_sentences = english_sentences

    def __len__(self):
        return len(self.english_sentences)
    
    def __getitem__(self, idx):
        return self.english_sentences[idx], self.fulfulde_sentences[idx]

In [ ]:
dataset = TextDataset(fulfulde_sentences, english_sentences)

In [ ]:
len(dataset)

In [ ]:
dataset[1]

In [ ]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [ ]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

In [ ]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=fulfulde_to_index[PADDING_TOKEN],
                                reduction="none")

for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda_is_available() else torch.device('cpu')

In [ ]:
NEG_INFTY = -1e9

def create_masks(eng_batch, fuh_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
        eng_sentence_length, fuh_sentence_length = len(eng_batch[idx]), len(fuh_batch[idx])
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
        fuh_chars_to_padding_mask = np.arange(fuh_sentence_length + 1, max_sequence_length)
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, fuh_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, fuh_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, fuh_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, fuh_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, fuh_batch)
        optim.zero_grad()
        fuh_predictions = transformer(
            eng_batch,
            fuh_batch,
            encoder_self_attention_mask.to(device),
            decoder_self_attention_mask.to(device),
            decoder_cross_attention_mask.to(device),
            enc_start_token = False,
            enc_end_token = False,
            dec_start_token = True,
            dec_end_token = True
        )
        labels = transformer.decoder.sentence_embedding.batch_tokenize(fuh_batch, start_token=False, end_token=True)
        loss = criterian(
            fuh_predictions.view(-1, fuh_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indices = torch.where(labels.view(-1) == fulfulde_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indices.sum()
        loss.backward()
        optim.step()

        if batch_num % 100 == 0:
            print(f"Iteration {batch_num}: {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"fulfulde Translation: {fuh_batch[0]}")
            fuh_sentence_predicted = torch.argmax(fuh_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in fuh_sentence_predicted:
                if idx == fulfulde_to_index[END_TOKEN]:
                    break
                predicted_sentence += index_to_fulfulde[idx.item()]
            print(f"fulfulde Prediction: {predicted_sentence}")

            transformer.eval()
            fuh_sentence = ("",)
            eng_sentence = ("i want to eat",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_sentence, fuh_sentence)
                predictions = transformer(
                    eng_sentence,
                    fuh_sentence,
                    encoder_self_attention_mask.to(device),
                    decoder_self_attention_mask.to(device),
                    decoder_cross_attention_mask.to(device),
                    enc_start_token = False,
                    enc_end_token = False,
                    dec_start_token = True,
                    dec_end_token = False
                )
                next_token_prob_distribution = predictions[0][word_counter]
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_fulfulde[next_token_index]
                fuh_sentence = (fuh_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                    break

            print(f"Evaluation translation (i want to eat) : {fuh_sentence}")
            print("--------------------------------------------------------")
